# Prediction Function
To do list
* Finding Effect of Each Feature
* Implament the prediction function

## The Effect of Each Feature in the Team Dataset on the Match's Result
* Using the neural network model, find out the contributions of all features on the result.
* Also review the academic articles on this topic

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.neural_network import MLPClassifier
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler

In [2]:
# load the data
team_df= pd.read_parquet('parquet_data/team_df_missing_handled.parquet')
player_df= pd.read_parquet('parquet_data/player_df_missing_handled.parquet')
# make a success colon to be predicted
team_df['Success'] = team_df['Result'].apply(
    lambda x: 1 if isinstance(x, str) and x.startswith('W') else (0 if isinstance(x, str) and x.startswith('L') else None))

In [3]:
# Özellikler ve hedef değişkeni ayırın
X = team_df[['Kills', 'Errors', 'Total Attacks', 'Hit Pct', 'Assists', 'SErr', 'Digs', 'Block Assists', 'PTS']]
y = team_df['Success']  # Sonuç (Kazanç: 1, Kaybetme: 0)

# Eğitim ve test verisine ayıralım
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sinir ağı modelini oluşturun ve eğitin
model = MLPClassifier(random_state=42)
model.fit(X_train, y_train)

# Permutation importance'ı hesaplayın
result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)

# Özelliklerin önemini yazdırın
importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': result.importances_mean
}).sort_values(by='Importance', ascending=False)

print(importance)

         Feature  Importance
8            PTS    0.297652
2  Total Attacks    0.219969
6           Digs    0.139202
0          Kills    0.065875
1         Errors    0.027493
4        Assists    0.026177
7  Block Assists    0.002659
5           SErr   -0.000212
3        Hit Pct   -0.000424


PTS (Points):
Açıklama: Bu, oyuncunun veya takımın kazandığı toplam puan sayısını gösterir. Voleybolda, her başarılı hücum, rakip takımın hatası, bloklar ve servis sayıları ile puan kazanılır.


Total Attacks (Toplam Hücumlar):
Açıklama: Bir oyuncunun veya takımın toplam hücum girişimleri sayısını belirtir. Voleybolda, hücumlar, topun rakip sahaya düzgün bir şekilde gönderilmesi için yapılan ataklardır.


Digs (Top Karşılama):
Açıklama: Rakip takımın hücumunu karşılama ve topu kontrol altında tutma işlemi. Digs, özellikle rakip hücumlarını savunurken başarılı olmayı ifade eder.


Kills (Sayılara Katkı):
Açıklama: Hücumların başarılı bir şekilde rakip saha içine düşürülmesidir. Yani, oyuncunun topu rakip sahaya atarak sayı kazanmasına kill denir. Bu, voleyboldaki en önemli hücum istatistiklerinden biridir.


Assists (Asistler):
Açıklama: Bir oyuncunun hücum yapmaya uygun bir topu, bir takım arkadaşına göndermesi işlemi. Yani, bir asist, rakip sahaya etkili bir hücum yapılmasına olanak sağlayan bir pas vermek olarak tanımlanabilir.


Errors (Hatalar):
Açıklama: Bir oyuncunun veya takımın, oyun kurallarına uygun olmayan bir şekilde topu düşürmesi. Hata yapıldığında rakip takıma bir puan verilir. Örneğin, top dışarıya atılabilir veya rakip blokta topu çıkarabilir.


Hit Pct (Hit Percentage / Hücum Yüzdesi):
Açıklama: Bu, oyuncunun veya takımın hücum başarısı oranını ifade eder. Hücum yüzdesi, başarılı hücumların toplam hücumlara oranı ile hesaplanır. Yüksek bir hücum yüzdesi, etkili hücumlar anlamına gelir.


SErr (Servis Hataları):
Açıklama: Bir oyuncunun yaptığı servis hataları. Örneğin, servis çizgisinden dışarıya top atmak veya rakibe doğrudan hata yaparak topu vermek. Bu hatalar genellikle oyun üzerindeki denetimi kaybetmeye yol açabilir.


Block Assists (Blok Asistleri):
Açıklama: Bir oyuncunun, bir takım arkadaşının blok girişimlerine yardım etmesi. Yani, blok asistleri, rakip oyuncunun hücumunun engellenmesine yardımcı olmaktır. Bu, takım savunmasının bir parçasıdır.

### Selected Features
* SErr'i ve Hit Pct'i eğitimlerimizde kullanmayacağız, çünkü hit pct iki feautre'ın oranı: Hit Percentage, başarılı hücumların toplam hücumlara oranı ile hesaplanır. SErr ise hiç etkili değil göürldüğü üzere.
* Kills özelliğini seçtik çünkü bir takım için maçı kazanmak önemli, nasıl kazandığı önemli değil. Maçı kazanıp kazanmaması ise voleybol sporunda sadece kazanılan sayıya göre belirleniyor. Bu yüzden her oyuncunun takıma kazandırdığı sayı olan kills özelliği oyuncun o takıma kattığı nihai değerdir.
* digs ve block assists savunma özellikleri, gayet iyi hocam kalsınlar.

## Function of Prediction of Players' Contributions

In [4]:
def predict_contribution(player_names):  # [kills, errors, total_attacks, assists, digs, block_assists, pts] kolonlarıan sahip tüm oyuncular
    target_columns =['Kills', 'Errors', 'Total Attacks', 'Assists','Digs','Block Assists', 'PTS']
    
    # Player_df'den her oyuncu için ortalama değerleri al
    player_avg_df = pd.DataFrame()

    for player_name in player_names:
        player_data = player_df[player_df['name'] == player_name]  # Oyuncuya ait verileri bul
        
        # Ortalama değerleri hesapla
        player_avg = player_data[target_columns].mean()
        
        # Bu ortalamayı tek bir satır olarak player_avg_df'e ekleyelim
        player_avg_df = pd.concat([player_avg_df, player_avg.to_frame().T], ignore_index=True)
    
    # Tahmin sonuçlarını saklamak için boş bir DataFrame
    predictions_df = pd.DataFrame()

    # Modeli yükleyip tahminleri almak
    for column in player_avg_df.columns:
        with open(f'pkl_files_regression/randomForest_model_{column}.pkl', 'rb') as f:
            loaded_model = pickle.load(f)
        
        # X (özellikler) ve y (hedef) verilerini ayıralım
        X = player_avg_df.drop(columns=column)  # Her hedef kolonu çıkararak X oluşturuyoruz
        
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)  # X_scaled = scaler.fit_transform(X)
        
        y_pred = loaded_model.predict(X_scaled)  # Tahmin işlemi
        
        # Tahminleri predictions_df'e ekleyelim
        predictions_df[column] = y_pred  # Tahmin sonuçlarını ilgili kolona ekliyoruz

    return player_avg_df, predictions_df

In [5]:
import numpy as np
X_new = pd.DataFrame({
    'Kills': [14.0],   
    'Errors': [3.0],   
    'Total Attacks': [37.0],   

    'Digs': [5.0],
    'Block Assists':[3.0],
    'PTS':[16.5]
})
player_avg_df, predictions_df=predict_contribution(['Carlson, Morgan', 'Doud, Madeleine'])
player_avg_df

,Kills,Errors,Total Attacks,Assists,Digs,Block Assists,PTS
0,11.0,3.0,25.571429,1.571429,6.714286,2.857143,12.714286
1,7.5,1.5,15.500000,1.666667,2.166667,2.666667,9.333333


In [6]:
predictions_df

,Kills,Errors,Total Attacks,Assists,Digs,Block Assists,PTS
0,23.789216,17.107608,27.174820,22.334376,10.440511,14.674549,22.299876
1,7.145391,5.737038,3.582101,90.544128,16.666273,23.573098,5.768222


# BU tahmin neye yarar?
Bi takım oluştururken koçlar ne yapar? iki tane kili yüksek adam alır, iki tane asisti yüksek adam alır vesaire, atıyorum tamamen. iki oyuncu arasında kararsız kaldığı zaman onların asisti en yüksek olanını almak çok düz mantık olur, hidden patternları koç anlamaz, görmez, bilmez. oyuncunun kill, eror, attack her şeyine bakıp asist 'oranını' hesaplayan bir ml'in ürettiği tahminler oyuncu seçiminde daha güçlü insight'lar verir. 
Sadece iki oyuncu arasında kalmakla da sınırlı değiliz. kills oranı açık ara farkla iyi olanı alırken tereddüt etmeden onu alırsın ama diyelim ki benzer kardeşim naabacan? diğer özelliklerin önemi burda devreye giriyor işte. diğer özellikleri de hesaba katarak iyi bir ratio yakalıyoruz. 